# 01 - 探索性数据分析与预处理 (EDA & Preprocessing)

本 notebook 完成以下任务：
1. 加载原始数据
2. 数据探索与可视化
3. 文本清洗与预处理
4. 保存处理后的数据

**优化内容**：
- 使用统一的配置文件和工具函数
- 添加随机种子设置
- 添加数据验证
- 优化文本清洗（可选择是否移除停用词）
- 添加日志记录

In [ ]:
# Cell 1: 环境设置与导入
import sys
import os

# 添加项目根目录到路径
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

# 导入项目配置和工具
import config
import utils

# 设置随机种子
utils.set_seed()

# 设置日志
logger = utils.setup_logger(
    'EDA_Preprocess',
    log_file=os.path.join(config.LOGS_DIR, 'eda_preprocess.log')
)

# 设置画图风格
sns.set(style="whitegrid")
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['figure.figsize'] = (10, 6)

# 显示环境信息
print("=" * 60)
print("环境配置信息")
print("=" * 60)
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"PyTorch Version: {torch.__version__}")
print(f"运行设备: {utils.get_device_info()}")
print(f"随机种子: {config.RANDOM_SEED}")
print("=" * 60)

logger.info("环境设置完成")

In [ ]:
# Cell 2: 加载原始数据
try:
    # 使用统一的文件查找函数
    data_path = utils.find_data_file(config.RAW_DATA_FILE)
    logger.info(f"找到数据文件: {data_path}")
    
    # 读取数据
    df = pd.read_csv(data_path)
    logger.info(f"数据加载成功，共 {len(df)} 条记录")
    
    print(f"\n✅ 数据加载成功！")
    print(f"📊 总样本数: {len(df):,}")
    print(f"📋 列名: {df.columns.tolist()}")
    print(f"💾 数据大小: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # 数据基本信息
    print("\n" + "=" * 60)
    print("数据基本信息")
    print("=" * 60)
    print(df.info())
    
    # 前5行预览
    print("\n" + "=" * 60)
    print("前 5 行数据预览")
    print("=" * 60)
    display(df.head())
    
except FileNotFoundError as e:
    logger.error(str(e))
    print(f"❌ 错误: {e}")
    raise

In [ ]:
# Cell 3: 数据验证与清洗
print("=" * 60)
print("数据质量检查")
print("=" * 60)

# 检查必需列
required_columns = [config.TEXT_COLUMN, config.LABEL_COLUMN]
try:
    utils.validate_dataframe(df, required_columns)
    logger.info("数据列验证通过")
    print("✅ 必需列验证通过")
except ValueError as e:
    logger.error(str(e))
    print(f"❌ {e}")
    raise

# 检查缺失值
missing_info = df[required_columns].isnull().sum()
print(f"\n📊 缺失值统计:")
print(missing_info)

if missing_info.sum() > 0:
    print(f"\n⚠️  发现 {missing_info.sum()} 个缺失值，将被删除")
    logger.warning(f"发现 {missing_info.sum()} 个缺失值")
    df = df.dropna(subset=required_columns)
    print(f"✅ 清洗后剩余 {len(df):,} 条记录")
    logger.info(f"删除缺失值后剩余 {len(df)} 条记录")
else:
    print("\n✅ 无缺失值")

# 检查重复值
duplicates = df.duplicated(subset=[config.TEXT_COLUMN]).sum()
print(f"\n📊 重复样本数: {duplicates:,} ({duplicates/len(df)*100:.2f}%)")
if duplicates > 0:
    logger.info(f"发现 {duplicates} 个重复样本")
    print("💡 提示: 可以选择是否删除重复项（本示例保留）")

# 检查文本长度分布
df['text_length'] = df[config.TEXT_COLUMN].str.len()
print(f"\n📊 文本长度统计:")
print(df['text_length'].describe())

In [ ]:
# Cell 4: 探索性数据分析 - 标签分布
print("=" * 60)
print("标签分布分析")
print("=" * 60)

# 统计各类别数量
label_counts = df[config.LABEL_COLUMN].value_counts()
print(f"\n{config.LABEL_COLUMN} 分布:")
for label, count in label_counts.items():
    percentage = count / len(df) * 100
    print(f"  {label:10s}: {count:7,} ({percentage:5.2f}%)")

logger.info(f"标签分布: {label_counts.to_dict()}")

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 柱状图
ax1 = axes[0]
bars = ax1.bar(label_counts.index, label_counts.values, 
               color=[config.SENTIMENT_COLORS.get(l.capitalize(), '#999999') 
                      for l in label_counts.index])
ax1.set_title('Sentiment Distribution (Count)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Sentiment', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)
ax1.grid(axis='y', alpha=0.3)

# 在柱子上显示数值
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}',
             ha='center', va='bottom', fontsize=10)

# 饼图
ax2 = axes[1]
colors = [config.SENTIMENT_COLORS.get(l.capitalize(), '#999999') 
          for l in label_counts.index]
wedges, texts, autotexts = ax2.pie(label_counts.values, 
                                     labels=label_counts.index,
                                     autopct='%1.1f%%',
                                     colors=colors,
                                     startangle=90)
ax2.set_title('Sentiment Distribution (Percentage)', fontsize=14, fontweight='bold')

# 美化百分比文字
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(11)
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.show()

# 检查类别不平衡
imbalance_ratio = label_counts.max() / label_counts.min()
if imbalance_ratio > 3:
    print(f"\n⚠️  数据存在类别不平衡 (最大类/最小类 = {imbalance_ratio:.2f})")
    print("💡 建议: 训练时考虑使用类别权重或重采样技术")
    logger.warning(f"类别不平衡比例: {imbalance_ratio:.2f}")
else:
    print(f"\n✅ 类别分布相对均衡 (比例: {imbalance_ratio:.2f})")

In [ ]:
# Cell 5: 文本长度分析
print("=" * 60)
print("文本长度分析")
print("=" * 60)

# 按标签统计文本长度
print("\n按情感类别统计文本长度:")
for label in df[config.LABEL_COLUMN].unique():
    subset = df[df[config.LABEL_COLUMN] == label]['text_length']
    print(f"\n{label.upper()}:")
    print(f"  平均长度: {subset.mean():.1f}")
    print(f"  中位数: {subset.median():.1f}")
    print(f"  标准差: {subset.std():.1f}")

# 可视化文本长度分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 总体分布
ax1 = axes[0]
ax1.hist(df['text_length'], bins=50, edgecolor='black', alpha=0.7)
ax1.axvline(df['text_length'].mean(), color='red', linestyle='--', 
            label=f'Mean: {df["text_length"].mean():.1f}')
ax1.axvline(df['text_length'].median(), color='green', linestyle='--',
            label=f'Median: {df["text_length"].median():.1f}')
ax1.set_title('Text Length Distribution (Overall)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Text Length (characters)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.legend()
ax1.grid(alpha=0.3)

# 按类别分布
ax2 = axes[1]
for label in df[config.LABEL_COLUMN].unique():
    subset = df[df[config.LABEL_COLUMN] == label]['text_length']
    color = config.SENTIMENT_COLORS.get(label.capitalize(), '#999999')
    ax2.hist(subset, bins=30, alpha=0.5, label=label, color=color, edgecolor='black')

ax2.set_title('Text Length Distribution by Sentiment', fontsize=14, fontweight='bold')
ax2.set_xlabel('Text Length (characters)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 6: 文本清洗
print("=" * 60)
print("文本清洗")
print("=" * 60)

# 清洗文本
print("\n⏳ 正在清洗文本...")
df[config.CLEANED_TEXT_COLUMN] = df[config.TEXT_COLUMN].apply(utils.clean_text)
logger.info("文本清洗完成")

# 可选：移除停用词（默认不移除，保留更多语义信息）
REMOVE_STOPWORDS = False  # 设置为 True 以移除停用词

if REMOVE_STOPWORDS:
    print("⏳ 正在移除停用词...")
    df[config.CLEANED_TEXT_COLUMN] = df[config.CLEANED_TEXT_COLUMN].apply(
        utils.remove_stopwords
    )
    logger.info("停用词移除完成")
    print("✅ 停用词已移除")
else:
    print("💡 保留停用词（更适合深度学习模型）")

print("\n✅ 文本清洗完成！")

# 展示清洗前后对比
print("\n" + "=" * 60)
print("清洗前后对比（前 3 个样本）")
print("=" * 60)

comparison_df = df[[config.TEXT_COLUMN, config.CLEANED_TEXT_COLUMN]].head(3)
for idx, row in comparison_df.iterrows():
    print(f"\n样本 {idx + 1}:")
    print(f"原文: {row[config.TEXT_COLUMN][:100]}...")
    print(f"清洗后: {row[config.CLEANED_TEXT_COLUMN][:100]}...")

# 检查清洗后是否有空文本
empty_texts = df[config.CLEANED_TEXT_COLUMN].str.strip().eq('').sum()
if empty_texts > 0:
    print(f"\n⚠️  发现 {empty_texts} 个空文本，将被删除")
    df = df[df[config.CLEANED_TEXT_COLUMN].str.strip() != '']
    logger.warning(f"删除 {empty_texts} 个空文本")
    print(f"✅ 清洗后剩余 {len(df):,} 条记录")
else:
    print("\n✅ 无空文本")

# 统计清洗后的文本长度
df['cleaned_text_length'] = df[config.CLEANED_TEXT_COLUMN].str.len()
print(f"\n📊 清洗后文本长度统计:")
print(df['cleaned_text_length'].describe())

In [ ]:
# Cell 7: 保存处理后的数据
print("=" * 60)
print("保存处理后的数据")
print("=" * 60)

# 删除临时列
df = df.drop(columns=['text_length', 'cleaned_text_length'], errors='ignore')

# 保存路径
save_path = os.path.join(config.DATA_DIR, config.PROCESSED_DATA_FILE)

# 保存数据
config.create_dirs()  # 确保目录存在
df.to_csv(save_path, index=False)

print(f"\n✅ 数据已保存到: {save_path}")
print(f"📊 保存记录数: {len(df):,}")
print(f"📋 保存列: {df.columns.tolist()}")
print(f"💾 文件大小: {os.path.getsize(save_path) / 1024**2:.2f} MB")

logger.info(f"数据已保存: {save_path}, 共 {len(df)} 条记录")

# 最终数据预览
print("\n" + "=" * 60)
print("最终数据预览")
print("=" * 60)
display(df.head())

print("\n" + "=" * 60)
print("✅ EDA 与预处理完成！")
print("=" * 60)
print("下一步: 运行 02_Baseline_Model.ipynb 训练基线模型")